In [1]:

import os
from queue import SimpleQueue
from utils.helpers import get_input

os.chdir('/Users/soup/Documents/projects/advent-of-code')

In [6]:
input = get_input(2015, 22)

Found a datafile!


40
41


TypeError: ord() expected a character, but string of length 10 found